In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from feature_hasher_transformer import FeatureHasherTransformer



# Load and preprocess data
df = pd.read_csv("data/train/cleaned_data.csv")

# Drop rows with missing or invalid target
df = df.dropna(subset=['material_price'])
df = df[df['material_price'] > 0]

# Fill missing text fields
text_columns = ['material_name', 'material_type', 'material_subtype',
                'surgeon_name', 'procedure_name']
df[text_columns] = df[text_columns].fillna('missing').astype(str)

# Combine categorical text fields
df['combined_features'] = df[text_columns].agg(' '.join, axis=1)

# Log-transform target
y_log = np.log(df['material_price'].values)

# Train/Test Split
X_train_text, X_test_text, y_train_log, y_test_log = train_test_split(
    df['combined_features'], y_log, test_size=0.2, random_state=42
)


# Build Pipeline
pipeline = Pipeline([
    ('hasher', FeatureHasherTransformer(n_features=100)),
    ('model', XGBRegressor(n_estimators=100, max_depth=6,
                           learning_rate=0.1, random_state=42))
])

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint


# Define hyperparameter space for XGBoost
param_distributions = {
    'model__n_estimators': randint(50, 300),
    'model__max_depth': randint(3, 10),
    'model__learning_rate': uniform(0.01, 0.3),
    'model__subsample': uniform(0.5, 0.5),
    'model__colsample_bytree': uniform(0.5, 0.5),
    'model__reg_alpha': uniform(0, 1),
    'model__reg_lambda': uniform(0, 1),
}


# Randomized Search
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=30,
    scoring='r2',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Fit randomized search
random_search.fit(X_train_text, y_train_log)

# Best model
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)


# Final Evaluation
y_pred_log = best_model.predict(X_test_text)
y_pred = np.exp(y_pred_log)
y_test = np.exp(y_test_log)


C:\Users\HP\AppData\Local\Temp\ipykernel_10696\744459975.py:13: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/train/cleaned_data.csv")


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Hyperparameters: {'model__colsample_bytree': np.float64(0.9244569121330419), 'model__learning_rate': np.float64(0.22651885634946198), 'model__max_depth': 9, 'model__n_estimators': 191, 'model__reg_alpha': np.float64(0.49379559636439074), 'model__reg_lambda': np.float64(0.5227328293819941), 'model__subsample': np.float64(0.7137705091792748)}


In [4]:
# === Step: Optimization Using Test Dataset ===

# Load the test dataset (same as Ridge workflow)
test_data = pd.read_csv('data/test/new_data.csv', low_memory=False)

# Preprocessing
text_columns = ['material_name', 'material_type', 'material_subtype',
                'surgeon_name', 'procedure_name']
test_data[text_columns] = test_data[text_columns].fillna('missing').astype(str)
test_data['combined_features'] = test_data[text_columns].agg(' '.join, axis=1)
test_data['is_default'] = (test_data['surgeon_name'] == 'Standardized').astype(int)

# Predict log prices and back-transform
log_preds = best_model.predict(test_data['combined_features'])
test_data['predicted_price'] = np.exp(log_preds)
test_data['predicted_price'] = np.clip(test_data['predicted_price'], 0, None)



# === Global minimum predicted price per material ===
global_min_predicted = test_data.groupby('material_name')['predicted_price'].min().to_dict()

# Use actual price if cheaper and available
def get_cheapest_price(row):
    predicted = global_min_predicted.get(row['material_name'], np.inf)
    if not np.isnan(row.get('material_price', np.nan)):
        return min(predicted, row['material_price'])
    return predicted

# Apply optimization
results = []
for proc_id in test_data['procedure_id'].unique():
    proc_data = test_data[test_data['procedure_id'] == proc_id].copy()

    default_data = proc_data[proc_data['is_default'] == 1]
    default_materials = set(default_data['material_name'])

    surgeon_data = proc_data[proc_data['is_default'] == 0]
    surgeon_added = set(surgeon_data[surgeon_data['surgeon_specific_action'] != 'default']['material_name']) - default_materials
    all_materials = default_materials.union(surgeon_added)

    optimized_materials = {}
    for mat in all_materials:
        mat_rows = proc_data[proc_data['material_name'] == mat]
        if not mat_rows.empty:
            # Get the first row as representative to use actual price if present
            cheapest_row = mat_rows.iloc[0]
            cheapest_price = get_cheapest_price(cheapest_row)
            optimized_materials[mat] = cheapest_price

    optimized_cost = sum(optimized_materials.values())

    for mat, price in optimized_materials.items():
        results.append({
            'procedure_id': proc_id,
            'material_name': mat,
            'optimized_price': price,
            'optimized_cost': optimized_cost
        })

# Save results to CSV
optimized_df = pd.DataFrame(results)
optimized_df.to_csv("xgboost_test_optimization_results.csv", index=False)
print("\nSaved: xgboost_test_optimization_results.csv")



Saved: xgboost_test_optimization_results.csv


In [5]:

# Clip predictions
y_pred_clipped = np.clip(y_pred, 0, None)
num_clipped = np.sum(y_pred < 0)

# Metrics
mse = mean_squared_error(y_test, y_pred_clipped)
mae = mean_absolute_error(y_test, y_pred_clipped)
r2 = r2_score(y_test, y_pred_clipped)
#rmse = mean_squared_error(y_test, y_pred_clipped, squared=False)

print("=== Tuned Model Evaluation ===")
print(f"R² Score: {r2:.2f}")
#print(f"RMSE:     {rmse:.2f}")
print(f"MSE:      {mse:.2f}")
print(f"MAE:      {mae:.2f}")
print(f"Clipped Predictions: {num_clipped}")


# Cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores_tuned = cross_val_score(best_model, df['combined_features'], y_log, cv=5, scoring='r2')
print(f"Tuned Model Cross-validated R²: {np.mean(cv_scores_tuned):.2f} ± {np.std(cv_scores_tuned):.2f}")

# Save results
results_df = pd.DataFrame({
    'Actual Price': y_test,
    'Predicted Price': y_pred_clipped
})
results_df.to_csv('predicted_vs_actual_prices.csv', index=False)
print("CSV file has been saved as 'predicted_vs_actual_prices.csv'.")


=== Tuned Model Evaluation ===
R² Score: 0.96
MSE:      471.91
MAE:      10.86
Clipped Predictions: 0
Tuned Model Cross-validated R²: 0.78 ± 0.04
CSV file has been saved as 'predicted_vs_actual_prices.csv'.


In [6]:
import joblib


joblib.dump({
    'model':best_model
}, 'models/model_weights.joblib')


['models/model_weights.joblib']